In [48]:
#import dataset

import tensorflow as tf 
import tensorflow_datasets as tfds
import io

In [23]:
#download data set
#imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

#load data setnya yang sudah didownload manual
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True, data_dir="./data/", download=False)



In [24]:
#tampilkan informasi tentang dataset
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='data\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <Spli

In [25]:
print(imdb)

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'unsupervised': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


### Preview Data

In [26]:
#with_info=True : menerima informasi tambahan dari dataset
#as_supervised=True : konversi data ke bentuk yang lebih sederhana (input, label) agar mudah digunakan dalam supervised learning
#download=False berarti kita tidak perlu mengunduh ulang dataset ini karena sudah tersedia secara lokal.

#melihat satu contoh ulasan
single_example=list(imdb['train'].take(1))[0]

#take(1) : mengambil 1 contoh 
# list()[0] : konversi jadi list dan ambil item pertama

#tampilkan hasil
print(single_example[0]) #kalau atasnya 0 berarti ini juga 0

# [1] = positive review
# [0] = negative review

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)


### Split Data

In [ ]:
# pisah(split) menjadi data train dan data test
#masing-masing ada 25000 data

# Memisahkan review dan label untuk dataset train dan test
train_data, test_data = imdb['train'], imdb['test']

#ambil review  dari data train tanpa melihat label
#map() : membaca setiap (review, label) dari data
train_reviews = train_data.map(lambda review, label:review)
train_labels = train_data.map(lambda review, label:label)

test_reviews = test_data.map(lambda review, label:review)
test_labels = test_data.map(lambda review, label:label)

In [28]:
#buat layer
vectorize_layer=tf.keras.layers.TextVectorization(max_tokens=10000)

vectorize_layer.adapt(train_reviews)


In [ ]:
#fungsi padding : memastikan setiap urutan teks memiliki panjang yang sama
def padding_func(sequences):
    #batch:sekelompok data yang diproses bersamaan
    
    #membuat batch  
    #sequences.cardinality(): menghitung berapa banyak item yang ada dalam dataset sequences
    sequences=sequences.ragged_batch(batch_size=sequences.cardinality())
    
    #mengambil satu batch (berisi banyak elemen data) dari data?
    sequences = sequences.get_single_element()
    
    padded_sequences=tf.keras.utils.pad_sequences(sequences.numpy(), #ubah bentuk data agar bisa diolah
                                                  maxlen=120, #panjang maksimal urutan 120
                                                  truncating='post', #memotong urutan dari belakang jika terlalu panjang
                                                  padding='pre') #menambahkan padding di depan jika terlalu pendek
    
    padded_sequences=tf.data.Dataset.from_tensor_slices(padded_sequences) 
    #tadi kan data diubah jadi numpy, sekarang diubah jadi tensorflow lagi
    
    # Padding dilakukan setelah sequence dibentuk oleh TextVectorization
    return padded_sequences

In [ ]:
#setelah ubah kalimat string menjadi integer, apply method padding untuk sama ratakan urutan

# Menerapkan vectorization dan padding
train_sequences = train_reviews.map(lambda text:vectorize_layer(text)).apply(padding_func)
test_sequences = test_reviews.map(lambda text:vectorize_layer(text)).apply(padding_func)

## Preparing Data

In [31]:
# menggabungkan train_sequences, train_labels menjadi (input, label) dalam satu dataset
#seperti membuat 2 kolom
train_dataset_vectorized=tf.data.Dataset.zip(train_sequences, train_labels)

test_dataset_vectorized=tf.data.Dataset.zip(test_sequences, train_labels)


In [ ]:
# Menyiapkan dataset untuk training
SHUFFLE_BUFFER_SIZE=1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE
BATCH_SIZE=32

train_dataset_final=(train_dataset_vectorized
                     .cache()
                     .shuffle(SHUFFLE_BUFFER_SIZE)
                     .prefetch(PREFETCH_BUFFER_SIZE)
                     .batch(BATCH_SIZE))

test_dataset_final=(test_dataset_vectorized
                     .cache()
                     .prefetch(PREFETCH_BUFFER_SIZE)
                     .batch(BATCH_SIZE))

In [ ]:
# Hyperparameters
VOCAB_SIZE = 10000
MAX_LENGTH = 120
EMBEDDING_DIM = 16
PADDING_TYPE = 'pre'
TRUNC_TYPE = 'post'

In [34]:
model=tf.keras.Sequential([
    tf.keras.Input(shape=(120,)),
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    
    #jadikan array 1D
    #tf.keras.layers.Flatten() ,
    tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

- flatten akan menghasilkan shape yang besar
- untuk alternatif, bisa menggunakan 
  tf.keras.layers.GlobalAveragePooling1D(),
  
  menghasilkan array 1D dengan shape lebih kecil yang lebih simple dan lebih cepat diproses.

Walaupun Flatten lebih lambat, tapi flatten lebih akurat

### Compile model

In [35]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 120, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │           102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,109 (625.43 KB)

 Trainable params: 160,109 (625.43 KB)

 Non-trainable params: 0 (0.00 B)

### Train Model

In [37]:
EPOCHS=10
model.fit(train_dataset_final,
          epochs=EPOCHS,
          validation_data=test_dataset_final)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6593 - loss: 0.6382 - val_accuracy: 0.4980 - val_loss: 0.9651
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8449 - loss: 0.3716 - val_accuracy: 0.4993 - val_loss: 1.2588
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8781 - loss: 0.2961 - val_accuracy: 0.4996 - val_loss: 1.4152
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8981 - loss: 0.2559 - val_accuracy: 0.4980 - val_loss: 1.5612
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9108 - loss: 0.2301 - val_accuracy: 0.4978 - val_loss: 1.6770
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9213 - loss: 0.2096 - val_accuracy: 0.4981 - val_loss: 1.7802
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9297 - loss: 0.1906 - val_accuracy: 0.4987 - val_loss: 1.8907
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9375 - loss: 0.1779 - val_accuracy: 0.

### Menggunakan Flatten

In [38]:
model2=tf.keras.Sequential([
    tf.keras.Input(shape=(120,)),
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    
    #jadikan array 1D
    tf.keras.layers.Flatten() ,
    #tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [40]:
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )

EPOCHS=10
model2.fit(train_dataset_final,
          epochs=EPOCHS,
          validation_data=test_dataset_final)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6096 - loss: 0.6219 - val_accuracy: 0.4959 - val_loss: 1.2499
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8725 - loss: 0.3028 - val_accuracy: 0.4978 - val_loss: 1.5472
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9426 - loss: 0.1630 - val_accuracy: 0.4994 - val_loss: 1.9402
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9896 - loss: 0.0534 - val_accuracy: 0.4987 - val_loss: 2.3640
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9954 - loss: 0.0234 - val_accuracy: 0.4984 - val_loss: 2.6876
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9981 - loss: 0.0110 - val_accuracy: 0.4981 - val_loss: 2.9397
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9991 - loss: 0.0062 - val_accuracy: 0.4986 - val_loss: 3.2662
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9996 - loss: 0.0023 - val_accuracy: 0.

#### Embedding

 Lapisan embedding berfungsi untuk mengonversi kata-kata atau token yang diinput menjadi representasi vektor berukuran tetap (dimensi embedding).

In [ ]:
#mengakses lapisan pertama (layers[0]) dari model, sebagai lapisan embedding?

# Mengambil bobot dari lapisan embedding
embedding_layer=model.layers[0]

#lapisan [0] : lapisan embedding
#lapisan [1] : lapisan dense
#lapisan [2] : lapisan output

#mengembil bobot dari lapisan embedding
embedding_weights =  embedding_layer.get_weights()[0]

print(embedding_weights.shape) #shape: (covab_size, embedding_dim)

(10000, 16)


Contoh

Jika vocab_size = 5000 dan embedding_dim = 64, maka ukuran dari embedding_weights akan menjadi (5000, 64), yang berarti terdapat 5000 kata dalam kosakata, masing-masing direpresentasikan oleh vektor embedding berdimensi 64.

### Plotting

In [52]:
#Membuka file dan menyimpan data

#vecs.tsv: simpan vector embedding dari setiap kata
#bukan dan write(bisa menulis di file)
out_v=io.open('vecs.tsv', 'w', encoding='utf-8')

#meta.tsv: menyimpan kata-kata atau "metadata"-nya.
out_m=io.open('meta.tsv', 'w', encoding='utf-8')

#mengambil semua kata di vocabulary dari vectorize_layer
vocabulary = vectorize_layer.get_vocabulary()

#Menulis Kata dan Vektor Embedding ke File

#iterasi kata pertama sampai terakhir di vocabulary
for word_num in range(1, len(vocabulary)):
   
    word_name = vocabulary[word_num] #Mengambil Nama Kata
    word_embedding = embedding_weights[word_num] #Mengambil Vektor Embedding Kata
    
    #Menulis ke File
    out_m.write(word_name + '\n') 
    out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")
    
out_v.close()
out_m.close()


- Mulai dari 1 agar tidak menuliskan "kata kosong" yang biasa ada di posisi 0 untuk padding.
- word_name berisi kata yang akan ditulis ke file meta.tsv
- word_embedding adalah array yang menyimpan representasi vektor dari word_name.
- Menulis word_name ke file meta.tsv di setiap baris, agar file ini berisi daftar kata.
- Menulis vektor word_embedding ke vecs.tsv. Setiap nilai dalam array word_embedding dipisahkan dengan \t (tab) agar vektor ini berbentuk tab-delimited pada setiap barisnya.

### Analogi
Anggap saja vocabulary ini seperti kamus di mana setiap kata punya definisi. Di sini, meta.tsv adalah daftar kata yang berfungsi seperti daftar isi buku, dan vecs.tsv adalah bagian yang memuat penjelasan atau deskripsi rinci (vektor embedding) dari setiap kata.

Contohnya, jika vocabulary berisi ["", "good", "bad", "excellent", "terrible"], maka:

- meta.tsv akan berisi:
  - good
  - bad
  - excellent
  - terrible
  
- vecs.tsv mungkin akan berisi sesuatu seperti:
  
  0.1    -0.4   0.6    ...   (embedding untuk "good")
  
  -0.3   0.8   -0.2    ...   (embedding untuk "bad")
 
  ...